<a href="https://colab.research.google.com/github/RobertGodin/CodePython/blob/master/Notebooks/LLM/ExempleReglageFinBertTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Exemple de réglage fin de BERT pour analyse de sentiment de TWEET
# Installation des bibliothèques
%pip install numpy
%pip install pandas
%pip install transformers
%pip install datasets
%pip install evaluate
%pip install torch
%pip install huggingface_hub
%pip install accelerate

In [20]:
# Chargement des données de tweet
nom_dataset = "mteb/tweet_sentiment_extraction"
import pandas as pd
from datasets import load_dataset
dataset_tweet = load_dataset(nom_dataset)
print(dataset_tweet)
df_ent = pd.DataFrame(dataset_tweet["train"])
df_ent.head(3)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})


,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative


In [21]:
# Chargement du modèle et tokenizer Bert de Google
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
nom_modele = 'google-bert/bert-base-uncased'
modele = BertForSequenceClassification.from_pretrained(nom_modele, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(nom_modele)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# Vectorisation des textes pour l'entrainement
def tokenize_text(data):
  return tokenizer(data["text"], padding = "max_length",truncation=True)
dataset_vectorise = dataset_tweet.map(tokenize_text, batched=True)
dataset_ent = dataset_vectorise["train"].shuffle(seed=42).select(range(500))  #petite collection de 100
dataset_test = dataset_vectorise["test"].shuffle(seed=42).select(range(500))
df_ent=pd.DataFrame(dataset_ent)
df_ent.head(3)

,id,text,label,label_text,input_ids,token_type_ids,attention_mask
0,fd8e3ede29,what happen to the maids of yours? Quit once ...,1,neutral,"[101, 2054, 4148, 2000, 1996, 29229, 1997, 673...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,9206b5221d,Listening to the commentary track on Holiday I...,2,positive,"[101, 5962, 2000, 1996, 8570, 2650, 2006, 6209...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,d6b51b539f,"chatting with some of my old classmates, helly...",1,neutral,"[101, 22331, 2007, 2070, 1997, 2026, 2214, 198...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [23]:
# Quelques exemples de prédictions avant entrainement
print("Predictions avant entrainement:")
print("--------------------------------------------------")
modele.to("cpu")
import torch
def predictions(liste_exemples):
  classe_etiquettes = {0: "negative", 1: "neutral", 2: "positive"}
  for texte in list_exemples :
    entrees = tokenizer.encode(texte, return_tensors="pt")
    logits = modele(entrees).logits
    predictions = torch.argmax(logits)
    sentiment = classe_etiquettes[predictions.item()]
    print("Analyse de sentiment pour '",texte,"' est :", sentiment)
liste_exemples = ["This is a very good comment.", "I don't recommend this restaurant.",
                   "This is better that most !", "Don't ever go there !", "This is a good and bad thing."]
predictions(liste_exemples)

Predictions avant entrainement:
--------------------------------------------------
Analyse de sentiment pour ' This is a very good comment. ' est : positive
Analyse de sentiment pour ' I don't recommend this restaurant. ' est : neutral
Analyse de sentiment pour ' This is better that most ! ' est : positive
Analyse de sentiment pour ' Don't ever go there ! ' est : positive
Analyse de sentiment pour ' This is a good and bad thing. ' est : positive


In [24]:
from transformers import Trainer, TrainingArguments
arguments_training = TrainingArguments(
    output_dir='dossier_bert_ft_tweet', # Dossier pour les résultats
    evaluation_strategy="epoch",

    num_train_epochs=10, # Défaut est 3
    learning_rate=2e-5, # Taux d'apprentissage
    per_device_train_batch_size=4, # Taille des mini-lots
    per_device_eval_batch_size=4,
    weight_decay=0.01, # Pour régularisation L2
)
import evaluate
import numpy as np
exactitude = evaluate.load("accuracy")
def calcul_metriques(p):
    logits, etiquettes = p
    predictions = np.argmax(logits, axis=1)
    return exactitude.compute(predictions=predictions, references=etiquettes)

entraineur = Trainer(
    model=modele,
    args=arguments_training,
    train_dataset=dataset_ent,
    eval_dataset=dataset_test,
    compute_metrics=calcul_metriques
)
entraineur.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.836173,0.630000
2,No log,0.667315,0.740000
3,No log,1.198087,0.724000
4,0.532900,1.476678,0.700000
5,0.532900,1.452668,0.734000
6,0.532900,1.558753,0.714000
7,0.532900,1.620657,0.718000
8,0.049000,1.708030,0.724000
9,0.049000,1.705059,0.722000
10,0.049000,1.724659,0.720000


TrainOutput(global_step=1250, training_loss=0.2345967887878418, metrics={'train_runtime': 723.9184, 'train_samples_per_second': 6.907, 'train_steps_per_second': 1.727, 'total_flos': 1315567088640000.0, 'train_loss': 0.2345967887878418, 'epoch': 10.0})

In [25]:
# Les prédictions après entrainement pour les exemples
print("Predictions après entrainement:")
print("--------------------------------------------------")
modele.to("cpu")
predictions(liste_exemples)

Predictions après entrainement:
--------------------------------------------------
Analyse de sentiment pour ' This is a very good comment. ' est : positive
Analyse de sentiment pour ' I don't recommend this restaurant. ' est : neutral
Analyse de sentiment pour ' This is better that most ! ' est : positive
Analyse de sentiment pour ' Don't ever go there ! ' est : negative
Analyse de sentiment pour ' This is a good and bad thing. ' est : negative


The sentiment for 'how on earth can I analyse this?' is:  negative
